In [3]:
import pandas as pd
import numpy as np
import joblib
import re

Load Dataset

In [4]:
df = pd.read_csv('../data/US_Accidents_March23_sample.csv')
df.head(10)

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-227993,Source2,2,2016-05-18 16:28:12,2016-05-18 16:58:12,39.815620,-82.822304,NaN,NaN,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
1,A-5584682,Source1,2,2021-10-17 00:25:30,2021-10-17 00:52:30,35.110082,-80.975220,35.107982,-80.978715,0.245,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-309630,Source2,2,2016-11-13 16:55:24,2016-11-13 17:40:24,47.234718,-122.487633,NaN,NaN,0.010,...,False,False,False,False,False,False,Night,Day,Day,Day
3,A-3580182,Source1,2,2017-03-15 09:21:23,2017-03-15 15:21:23,43.176570,-71.611080,43.177915,-71.624505,0.683,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-6523880,Source1,2,2021-02-12 04:27:00,2021-02-12 08:44:00,41.944932,-83.555199,41.945482,-83.559469,0.223,...,False,False,False,False,False,False,Night,Night,Night,Night
5,A-7100647,Source1,2,2020-06-19 16:26:01,2020-06-19 16:55:16,43.613860,-116.262640,43.613860,-116.262640,0.000,...,False,False,False,False,False,False,Day,Day,Day,Day
6,A-4190196,Source1,2,2022-02-22 09:02:46,2022-02-22 11:34:23,25.732933,-80.340203,25.733035,-80.335226,0.310,...,False,True,False,False,False,False,Day,Day,Day,Day
7,A-5962252,Source1,2,2021-08-28 21:19:00,2021-08-28 23:32:17,32.786155,-96.812695,32.783963,-96.813574,0.160,...,False,False,False,False,False,False,Night,Night,Night,Day
8,A-4658270,Source1,2,2022-06-19 10:50:00,2022-06-19 12:08:23,41.157002,-76.797775,41.175349,-76.791129,1.314,...,False,False,False,False,False,False,Day,Day,Day,Day
9,A-2099924,Source2,2,2019-05-13 06:15:11,2019-05-13 07:47:15,34.054260,-118.237823,NaN,NaN,0.000,...,False,False,False,False,True,False,Day,Day,Day,Day


In [5]:
# number of duplicated data
df.duplicated().sum()

0

In [6]:
df.columns

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [7]:
# drop columns that are not useful
not_useful = ['ID', 'Source', 'End_Time', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Weather_Timestamp']
df = df.drop(not_useful, axis=1)
df.columns

Index(['Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Street', 'City',
       'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [8]:
categorical_columns = [
    'Severity', 'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
    'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump',
    'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout',
    'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
    'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'
]

numerical_columns = [
    'Start_Lat', 'Start_Lng', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
    'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)'
]


In [9]:
for i in categorical_columns:  
  print(i,df[i].unique().size)

Severity 4
Street 39189
City 6976
County 1359
State 49
Zipcode 45862
Country 1
Timezone 5
Airport_Code 1706
Wind_Direction 25
Weather_Condition 89
Amenity 2
Bump 2
Crossing 2
Give_Way 2
Junction 2
No_Exit 2
Railway 2
Roundabout 2
Station 2
Stop 2
Traffic_Calming 2
Traffic_Signal 2
Turning_Loop 1
Sunrise_Sunset 3
Civil_Twilight 3
Nautical_Twilight 3
Astronomical_Twilight 3


In [10]:
# drop country and turning loop because they have only one unique value
df = df.drop(['Country','Turning_Loop'], axis=1)

In [11]:
print("All wind Direction: ", df['Wind_Direction'].unique())

All wind Direction:  ['North' 'NNW' 'SSW' 'WSW' 'NE' 'E' 'ESE' 'NW' 'CALM' 'SE' 'ENE' 'West'
 'SW' 'N' nan 'VAR' 'South' 'SSE' 'S' 'Calm' 'NNE' 'WNW' 'W' 'Variable'
 'East']


In [12]:
# simplify wind directions
direction_mapping = {
    'Calm': 'CALM',
    'West': 'W', 'WSW': 'W', 'WNW': 'W',
    'South': 'S', 'SSW': 'S', 'SSE': 'S',
    'North': 'N', 'NNW': 'N', 'NNE': 'N',
    'East': 'E', 'ESE': 'E', 'ENE': 'E',
    'Variable': 'VAR'
}

# apply the mapping to the 'Wind_Direction' column
df['Wind_Direction'] = df['Wind_Direction'].map(direction_mapping).fillna(df['Wind_Direction'])
print("All wind directions after simplification:", df['Wind_Direction'].unique())

All wind directions after simplification: ['N' 'S' 'W' 'NE' 'E' 'NW' 'CALM' 'SE' 'SW' nan 'VAR']


In [13]:
# combine unique weather conditions into a single string, separating with '!'
weather_conditions_str = '!'.join(df['Weather_Condition'].dropna().unique().tolist())

# split the string using specified delimiters and filter out repetitive elements
unique_weather_conditions = np.unique(np.array(re.split(
    r"!|\s/\s|\sand\s|\swith\s|Partly\s|Mostly\s|Blowing\s|Freezing\s", weather_conditions_str))).tolist()
print("Weather Conditions:", unique_weather_conditions)

Weather Conditions: ['', 'Clear', 'Cloudy', 'Drizzle', 'Dust', 'Dust Whirlwinds', 'Fair', 'Fog', 'Funnel Cloud', 'Hail', 'Haze', 'Heavy ', 'Heavy Drizzle', 'Heavy Rain', 'Heavy Snow', 'Heavy T-Storm', 'Heavy Thunderstorms', 'Ice Pellets', 'Light ', 'Light Drizzle', 'Light Ice Pellets', 'Light Rain', 'Light Rain Shower', 'Light Rain Showers', 'Light Sleet', 'Light Snow', 'Light Snow Shower', 'Light Thunderstorms', 'Mist', 'N/A Precipitation', 'Overcast', 'Patches of Fog', 'Rain', 'Rain Shower', 'Rain Showers', 'Sand', 'Scattered Clouds', 'Shallow Fog', 'Showers in the Vicinity', 'Sleet', 'Small Hail', 'Smoke', 'Snow', 'Squalls', 'T-Storm', 'Thunder', 'Thunder in the Vicinity', 'Thunderstorm', 'Thunderstorms', 'Volcanic Ash', 'Widespread Dust', 'Windy', 'Wintry Mix']


In [14]:
# map weather conditions to columns
weather_conditions = {
    'Clear': 'Clear',
    'Cloud': 'Cloud|Overcast',
    'Rain': 'Rain|storm',
    'Heavy_Rain': 'Heavy Rain|Rain Shower|Heavy T-Storm|Heavy Thunderstorms',
    'Snow': 'Snow|Sleet|Ice',
    'Heavy_Snow': 'Heavy Snow|Heavy Sleet|Heavy Ice Pellets|Snow Showers|Squalls',
    'Fog': 'Fog'
}

# apply conditions and create new columns
for column, condition in weather_conditions.items():
    df[column] = df['Weather_Condition'].str.contains(condition, case=False, na=False)


In [15]:
weather_conditions = ['Clear', 'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow', 'Fog']

for condition in weather_conditions:
    null_condition = df['Weather_Condition'].isnull()
    df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
    df[condition] = df[condition].astype(bool)

df.loc[:, ['Weather_Condition'] + weather_conditions]

df = df.drop('Weather_Condition', axis=1)


/tmp/ipykernel_239076/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
/tmp/ipykernel_239076/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[null_condition, condition] = df.loc[null_condition, 'Weather_Condition']
/tmp/ipykernel_239076/1656484887.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[null_

In [16]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'])

df['Year'] = df['Start_Time'].dt.year
df['Month'] = df['Start_Time'].dt.month
df['Day'] = df['Start_Time'].dt.day
df['Hour'] = df['Start_Time'].dt.hour
df['Minute']=df['Hour']*60.0+df["Start_Time"].dt.minute

df.loc[:4,['Start_Time', 'Year', 'Month',  'Day', 'Hour', 'Minute']]
df = df.drop('Start_Time', axis=1)

In [17]:
missing_values = pd.DataFrame(df.isnull().sum(), columns=['Missing_Count']).reset_index()
missing_values.columns = ['Feature', 'Missing_Percent(%)']
missing_values['Missing_Percent(%)'] = missing_values['Missing_Percent(%)'] / df.shape[0] * 100
missing_values = missing_values[missing_values['Missing_Percent(%)'] > 0]
missing_values


,Feature,Missing_Percent(%)
3,Street,0.113409
4,City,0.002283
7,Zipcode,0.025117
8,Timezone,0.108842
9,Airport_Code,0.304453
10,Temperature(F),2.182170
11,Wind_Chill(F),25.856465
12,Humidity(%),2.303951
13,Pressure(in),1.854121
14,Visibility(mi),2.337441


In [18]:
# dropping wind chill because it will be hard to impute since it is missing 25% of the data
df = df.drop('Wind_Chill(F)', axis=1)

In [19]:
df['Precipitation_NA'] = 0
df.loc[df['Precipitation(in)'].isnull(), 'Precipitation_NA'] = 1
df['Precipitation(in)'] = df['Precipitation(in)'].fillna(df['Precipitation(in)'].median())
df.loc[:5, ['Precipitation(in)', 'Precipitation_NA']]

,Precipitation(in),Precipitation_NA
0,0.0,1
1,0.0,0
2,0.0,1
3,0.0,1
4,0.0,0
5,0.0,0


In [20]:
df = df.dropna(subset=['Street','City','Zipcode','Airport_Code',
                       'Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight', 'Wind_Direction'])

In [21]:
# impute missing values with median
for column in ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']:
    df[column] = df[column].fillna(df[column].median())

In [22]:
df.isnull().sum().sum()

0

In [23]:
df.columns

Index(['Severity', 'Start_Lat', 'Start_Lng', 'Street', 'City', 'County',
       'State', 'Zipcode', 'Timezone', 'Airport_Code', 'Temperature(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Sunrise_Sunset',
       'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Clear',
       'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow', 'Fog', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Precipitation_NA'],
      dtype='object')

In [24]:
# save the cleaned data
df.to_csv('../data/US_Accidents_March23_cleaned.csv', index=False)